# Visited-Only Ablation: Prior Inclusion in Bayesian MCTS Backup

This notebook tests whether unvisited children's prior beliefs should be included during backup.

## Background

`aggregate_children()` is called in two contexts:

1. **At expansion** (default `visited_only=False`): All newly-created children are included
2. **During backup** (currently `visited_only=True`): Only visited children are aggregated

## Hypothesis

Including unvisited children's priors (`visited_only=False` in backup) might:
- **Help** when search is shallow (many unvisited children with informative NN priors)
- **Hurt** when unvisited children have high variance that dilutes the signal

## Experiments

1. **visited_only=True vs False** in backup step
2. **Hybrid approaches**: Weight by visit count, or only include high-prior unvisited children
3. **Varying simulation counts**: See if optimal setting changes with search depth

**Setup:** Use `Runtime > Change runtime type > GPU` for best performance.

In [ ]:
# Check GPU
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU: {gpu_name}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Install Rust toolchain
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ["PATH"] = f"{os.environ['HOME']}/.cargo/bin:" + os.environ["PATH"]

# Verify Rust installation
!rustc --version

In [ ]:
# Clone repository
!git clone https://github.com/caldred/nanozero.git
%cd nanozero

# Install Python dependencies
!pip install -q numpy scipy maturin

# Build and install Rust extension
%cd nanozero-mcts-rs
!maturin build --release
!pip install target/wheels/nanozero_mcts_rs-*.whl
%cd ..

# Verify Rust backend is available
!python -c "from nanozero.game import HAS_RUST_GAMES; print(f'Rust backend available: {HAS_RUST_GAMES}')"

---
## Train Connect4 Model

First, train a model we'll use for all ablation tests.

In [ ]:
# Train Connect4 - A100 optimized settings
# ~5-10 minutes on A100
!python -m scripts.train \
    --game=connect4 \
    --n_layer=4 \
    --num_iterations=150 \
    --games_per_iteration=64 \
    --training_steps=200 \
    --mcts_simulations=100 \
    --batch_size=256 \
    --buffer_size=100000 \
    --parallel_games=128 \
    --eval_interval=25

---
## Modified Bayesian MCTS with Configurable visited_only

Create variants that use different `visited_only` strategies during backup:

1. **visited_only=True** (current): Only aggregate visited children
2. **visited_only=False**: Include all children's beliefs
3. **Hybrid**: Weight by (visits + 1) pseudocount

In [ ]:
import math
import numpy as np
import torch
from typing import Dict, List, Optional, Tuple
import sys
sys.path.insert(0, '.')

from nanozero.config import BayesianMCTSConfig
from nanozero.game import Game
from nanozero.mcts import TranspositionTable


def normal_cdf(x: float) -> float:
    """Standard normal CDF using error function."""
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))


class BayesianNodeVisitedOnly:
    """
    MCTS tree node with Gaussian belief over value.
    Modified to support different visited_only strategies in aggregation.
    """

    def __init__(self, prior: float = 0.0, mu: float = 0.0, sigma_sq: float = 1.0):
        self.prior = prior
        self.mu = mu
        self.sigma_sq = sigma_sq
        self.children: Dict[int, 'BayesianNodeVisitedOnly'] = {}
        self.agg_mu: Optional[float] = None
        self.agg_sigma_sq: Optional[float] = None
        self.visits: int = 0

    def expanded(self) -> bool:
        return len(self.children) > 0

    def sample(self) -> float:
        return np.random.normal(self.mu, math.sqrt(self.sigma_sq))

    def update(self, value: float, obs_var: float, min_var: float = 1e-6) -> None:
        precision_prior = 1.0 / max(self.sigma_sq, min_var)
        precision_obs = 1.0 / max(obs_var, min_var)
        new_precision = precision_prior + precision_obs
        self.mu = (precision_prior * self.mu + precision_obs * value) / new_precision
        self.sigma_sq = max(1.0 / new_precision, min_var)

    def precision(self) -> float:
        return 1.0 / self.sigma_sq

    def aggregate_children(
        self, 
        prune_threshold: float = 0.01, 
        visited_only: bool = False,
        visit_weighted: bool = False  # NEW: weight by visits instead of binary filter
    ) -> None:
        """
        Compute aggregated belief from children using optimality weights.
        
        Args:
            prune_threshold: Children with P(optimal) < threshold get weight 0
            visited_only: If True, only aggregate children that have been visited
            visit_weighted: If True, weight children by (visits + 1) pseudocount
        """
        if not self.children:
            return

        if visited_only:
            children = [c for c in self.children.values() if c.visits > 0]
            if not children:
                return
        else:
            children = list(self.children.values())

        n = len(children)

        if n == 1:
            child = children[0]
            self.agg_mu = -child.mu
            self.agg_sigma_sq = child.sigma_sq
            return

        # Get child beliefs from parent's perspective (negate child values)
        mus = np.array([-c.mu for c in children])
        sigma_sqs = np.array([c.sigma_sq for c in children])

        # Find leader and challenger by mean
        sorted_idx = np.argsort(mus)[::-1]
        leader_idx = sorted_idx[0]
        challenger_idx = sorted_idx[1]

        # Compute optimality scores via pairwise Gaussian CDF comparisons
        scores = np.zeros(n)
        mu_L, sigma_sq_L = mus[leader_idx], sigma_sqs[leader_idx]
        mu_C, sigma_sq_C = mus[challenger_idx], sigma_sqs[challenger_idx]

        for i in range(n):
            if i == leader_idx:
                diff = mu_L - mu_C
                std = math.sqrt(sigma_sq_L + sigma_sq_C)
            else:
                diff = mus[i] - mu_L
                std = math.sqrt(sigma_sqs[i] + sigma_sq_L)

            if std > 1e-10:
                scores[i] = normal_cdf(diff / std)
            else:
                scores[i] = 1.0 if diff > 0 else 0.0

        # Apply visit weighting if enabled
        if visit_weighted:
            # Weight by (visits + 1) to give unvisited children some weight
            visit_weights = np.array([c.visits + 1 for c in children], dtype=np.float32)
            scores = scores * visit_weights

        # Soft prune and normalize
        scores[scores < prune_threshold] = 0.0
        total = scores.sum()
        if total < 1e-10:
            weights = np.ones(n) / n
        else:
            weights = scores / total

        # Aggregated mean (weighted average of children)
        self.agg_mu = float(np.sum(weights * mus))

        # Aggregated variance (squared weights + disagreement term)
        disagreement = (mus - self.agg_mu) ** 2
        self.agg_sigma_sq = float(np.sum(weights ** 2 * (sigma_sqs + disagreement)))


class BayesianMCTSVisitedOnly:
    """
    Bayesian MCTS with configurable visited_only strategy during backup.
    
    backup_mode options:
    - 'visited': Only aggregate visited children (current default)
    - 'all': Aggregate all children including unvisited
    - 'weighted': Aggregate all children weighted by (visits + 1)
    """

    def __init__(
        self,
        game: Game,
        config: BayesianMCTSConfig,
        backup_mode: str = 'visited',  # 'visited', 'all', or 'weighted'
        use_transposition_table: bool = True
    ):
        self.game = game
        self.config = config
        self.backup_mode = backup_mode
        self.use_tt = use_transposition_table
        self.tt = TranspositionTable(game) if use_transposition_table else None

    def clear_cache(self):
        if self.tt:
            self.tt.clear()

    def search(
        self,
        states: np.ndarray,
        model: torch.nn.Module,
        num_simulations: Optional[int] = None
    ) -> np.ndarray:
        if num_simulations is None:
            num_simulations = self.config.num_simulations

        num_states = states.shape[0]
        device = next(model.parameters()).device
        policies = np.zeros((num_states, self.game.config.action_size), dtype=np.float32)

        # Handle terminal states
        non_terminal_indices = []
        non_terminal_states = []
        for i in range(num_states):
            if self.game.is_terminal(states[i]):
                legal = self.game.legal_actions(states[i])
                if legal:
                    for a in legal:
                        policies[i, a] = 1.0 / len(legal)
            else:
                non_terminal_indices.append(i)
                non_terminal_states.append(states[i])

        if not non_terminal_states:
            return policies

        # Batch expand roots
        non_terminal_states_arr = np.stack(non_terminal_states)
        roots, _ = self._batch_expand_roots(non_terminal_states_arr, model, device)

        # Simulation loop
        for sim in range(num_simulations):
            leaves_to_expand = []
            terminal_backups = []
            expansion_backups = []

            for local_idx in range(len(roots)):
                root = roots[local_idx]
                state = non_terminal_states[local_idx]
                leaf_node, search_path, leaf_state, is_terminal = self._select_to_leaf(root, state)

                if is_terminal:
                    value = self.game.terminal_reward(leaf_state)
                    terminal_backups.append((local_idx, search_path, value))
                elif not leaf_node.expanded():
                    leaves_to_expand.append((local_idx, leaf_node, leaf_state))
                    expansion_backups.append((local_idx, search_path))

            # Batch expand
            if leaves_to_expand:
                nodes = [item[1] for item in leaves_to_expand]
                leaf_states = [item[2] for item in leaves_to_expand]
                values = self._batch_expand_leaves(nodes, leaf_states, model, device)

                for (local_idx, search_path), value in zip(expansion_backups, values):
                    self._backup(search_path, value)

            for local_idx, search_path, value in terminal_backups:
                self._backup(search_path, value)

        # Extract policies
        for local_idx, state_idx in enumerate(non_terminal_indices):
            policies[state_idx] = self._get_policy(roots[local_idx])

        return policies

    def _select_to_leaf(
        self,
        root: BayesianNodeVisitedOnly,
        state: np.ndarray
    ) -> Tuple[BayesianNodeVisitedOnly, List[Tuple[BayesianNodeVisitedOnly, int]], np.ndarray, bool]:
        node = root
        search_path = []
        current_state = state.copy()

        while node.expanded() and not self.game.is_terminal(current_state):
            action, child = self._select_child_thompson_ids(node)
            search_path.append((node, action))
            current_state = self.game.next_state(current_state, action)
            node = child

        is_terminal = self.game.is_terminal(current_state)
        return node, search_path, current_state, is_terminal

    def _create_children_from_policy(
        self,
        node: BayesianNodeVisitedOnly,
        state: np.ndarray,
        policy: np.ndarray,
        value: float
    ) -> None:
        legal_actions = self.game.legal_actions(state)
        sigma_0 = self.config.sigma_0

        eps = 1e-8
        legal_probs = np.array([policy[a] for a in legal_actions])
        legal_probs = legal_probs / (legal_probs.sum() + eps)
        log_probs = np.log(legal_probs + eps)
        entropy = -np.sum(legal_probs * log_probs)
        scale = sigma_0 * (math.sqrt(6) / math.pi)

        for i, action in enumerate(legal_actions):
            mu = -value - scale * (log_probs[i] + entropy)
            sigma_sq = sigma_0 ** 2
            node.children[action] = BayesianNodeVisitedOnly(
                prior=policy[action],
                mu=mu,
                sigma_sq=sigma_sq
            )

    def _batch_expand_leaves(
        self,
        nodes: List[BayesianNodeVisitedOnly],
        states: List[np.ndarray],
        model: torch.nn.Module,
        device: torch.device
    ) -> List[float]:
        batch_size = len(nodes)
        if batch_size == 0:
            return []

        results = [None] * batch_size
        miss_indices = []
        miss_canonical_keys = []

        for i in range(batch_size):
            state = states[i]
            node = nodes[i]

            if self.tt:
                cached = self.tt.get(state)
                if cached is not None:
                    policy, value = cached
                    self._create_children_from_policy(node, state, policy, value)
                    # At expansion: always use all children (visited_only=False)
                    node.aggregate_children(self.config.prune_threshold)
                    results[i] = value
                    continue

            if self.tt:
                canonical_key, _ = self.tt._canonical_key(state)
                miss_canonical_keys.append(canonical_key)
            miss_indices.append(i)

        if miss_indices:
            if self.tt and miss_canonical_keys:
                unique_keys = {}
                for j, (idx, key) in enumerate(zip(miss_indices, miss_canonical_keys)):
                    if key not in unique_keys:
                        unique_keys[key] = j
                unique_local_indices = list(unique_keys.values())
            else:
                unique_local_indices = list(range(len(miss_indices)))

            unique_states = [states[miss_indices[j]] for j in unique_local_indices]

            state_tensors = torch.stack([
                self.game.to_tensor(self.game.canonical_state(s)) for s in unique_states
            ]).to(device)

            action_masks = torch.stack([
                torch.from_numpy(self.game.legal_actions_mask(s))
                for s in unique_states
            ]).float().to(device)

            policies, values = model.predict(state_tensors, action_masks)
            policies = policies.cpu().numpy()
            values = values.cpu().numpy().flatten()

            if self.tt:
                for j, state in enumerate(unique_states):
                    self.tt.put(state, policies[j], values[j])

            for idx in miss_indices:
                state = states[idx]
                node = nodes[idx]

                if self.tt:
                    policy, value = self.tt.get(state)
                else:
                    local_idx = miss_indices.index(idx)
                    policy = policies[local_idx]
                    value = values[local_idx]

                self._create_children_from_policy(node, state, policy, value)
                # At expansion: always use all children
                node.aggregate_children(self.config.prune_threshold)
                results[idx] = value

        return results

    def _batch_expand_roots(
        self,
        states: np.ndarray,
        model: torch.nn.Module,
        device: torch.device
    ) -> Tuple[List[BayesianNodeVisitedOnly], np.ndarray]:
        batch_size = states.shape[0]
        roots = [BayesianNodeVisitedOnly() for _ in range(batch_size)]

        cache_hits = [None] * batch_size
        miss_indices = []

        if self.tt:
            for i in range(batch_size):
                cached = self.tt.get(states[i])
                if cached is not None:
                    cache_hits[i] = cached
                else:
                    miss_indices.append(i)
        else:
            miss_indices = list(range(batch_size))

        if miss_indices:
            miss_states = np.stack([states[i] for i in miss_indices])

            state_tensors = torch.stack([
                self.game.to_tensor(self.game.canonical_state(s)) for s in miss_states
            ]).to(device)

            action_masks = torch.stack([
                torch.from_numpy(self.game.legal_actions_mask(s))
                for s in miss_states
            ]).float().to(device)

            policies, values = model.predict(state_tensors, action_masks)
            policies = policies.cpu().numpy()
            values = values.cpu().numpy().flatten()

            for j, idx in enumerate(miss_indices):
                if self.tt:
                    self.tt.put(states[idx], policies[j], values[j])
                cache_hits[idx] = (policies[j], values[j])

        all_values = np.zeros(batch_size, dtype=np.float32)
        for i, root in enumerate(roots):
            policy, value = cache_hits[i]
            self._create_children_from_policy(root, states[i], policy, value)
            # At expansion: always use all children
            root.aggregate_children(self.config.prune_threshold)
            all_values[i] = value

        return roots, all_values

    def _select_child_thompson_ids(
        self,
        node: BayesianNodeVisitedOnly
    ) -> Tuple[int, BayesianNodeVisitedOnly]:
        children = list(node.children.items())
        if len(children) == 1:
            return children[0]

        samples = [(action, child, -child.sample()) for action, child in children]
        samples.sort(key=lambda x: x[2], reverse=True)

        leader_action, leader_node, _ = samples[0]
        challenger_action, challenger_node, _ = samples[1]

        alpha = self.config.ids_alpha
        precision_i = leader_node.precision()
        precision_j = challenger_node.precision()

        beta = (precision_i + alpha) / (precision_i + precision_j + 2 * alpha)

        if np.random.random() < beta:
            return challenger_action, challenger_node
        else:
            return leader_action, leader_node

    def _backup(
        self,
        search_path: List[Tuple[BayesianNodeVisitedOnly, int]],
        leaf_value: float
    ) -> None:
        # Determine aggregation parameters based on backup_mode
        if self.backup_mode == 'visited':
            visited_only = True
            visit_weighted = False
        elif self.backup_mode == 'all':
            visited_only = False
            visit_weighted = False
        elif self.backup_mode == 'weighted':
            visited_only = False
            visit_weighted = True
        else:
            raise ValueError(f"Unknown backup_mode: {self.backup_mode}")

        for i, (parent, action) in enumerate(reversed(search_path)):
            child = parent.children[action]
            child.visits += 1

            if i == 0:
                if not child.expanded():
                    child.update(leaf_value, self.config.obs_var, self.config.min_variance)
                else:
                    if child.agg_mu is not None:
                        child.mu = child.agg_mu
                        child.sigma_sq = child.agg_sigma_sq

            # Use configurable backup mode
            parent.aggregate_children(
                self.config.prune_threshold, 
                visited_only=visited_only,
                visit_weighted=visit_weighted
            )

            if parent.agg_mu is not None:
                parent.mu = parent.agg_mu
                parent.sigma_sq = parent.agg_sigma_sq

    def _get_policy(self, root: BayesianNodeVisitedOnly) -> np.ndarray:
        policy = np.zeros(self.game.config.action_size, dtype=np.float32)

        if not root.expanded():
            return policy

        actions = list(root.children.keys())
        children = [root.children[a] for a in actions]
        n = len(children)

        if n == 1:
            policy[actions[0]] = 1.0
            return policy

        mus = np.array([-c.mu for c in children])
        sigma_sqs = np.array([c.sigma_sq for c in children])

        sorted_idx = np.argsort(mus)[::-1]
        leader_idx = sorted_idx[0]
        challenger_idx = sorted_idx[1]

        scores = np.zeros(n)
        mu_L, sigma_sq_L = mus[leader_idx], sigma_sqs[leader_idx]
        mu_C, sigma_sq_C = mus[challenger_idx], sigma_sqs[challenger_idx]

        for i in range(n):
            if i == leader_idx:
                diff = mu_L - mu_C
                std = math.sqrt(sigma_sq_L + sigma_sq_C)
            else:
                diff = mus[i] - mu_L
                std = math.sqrt(sigma_sqs[i] + sigma_sq_L)

            if std > 1e-10:
                scores[i] = normal_cdf(diff / std)
            else:
                scores[i] = 1.0 if diff > 0 else 0.0

        total = scores.sum()
        if total < 1e-10:
            for action in actions:
                policy[action] = 1.0 / n
        else:
            for i, action in enumerate(actions):
                policy[action] = scores[i] / total

        return policy


print("BayesianMCTSVisitedOnly class defined successfully.")
print("Backup modes: 'visited' (current), 'all' (include unvisited), 'weighted' (visit-weighted)")

---
## Arena Test Function

In [ ]:
from nanozero.game import get_game
from nanozero.model import AlphaZeroTransformer
from nanozero.mcts import BatchedMCTS
from nanozero.common import sample_action, load_checkpoint
from nanozero.config import get_model_config, MCTSConfig, BayesianMCTSConfig


def run_arena(game, model, puct_mcts, ttts_mcts, num_games, mcts_simulations):
    """
    Run arena between PUCT and TTTS, return results from TTTS perspective.
    
    Returns:
        Tuple of (wins, draws, losses) for TTTS
    """
    wins, draws, losses = 0, 0, 0

    for i in range(num_games):
        state = game.initial_state()
        ttts_turn = 1 if i % 2 == 0 else -1  # Alternate who goes first

        while not game.is_terminal(state):
            current = game.current_player(state)
            if current == ttts_turn:
                policy = ttts_mcts.search(
                    state[np.newaxis, ...], model,
                    num_simulations=mcts_simulations
                )[0]
            else:
                policy = puct_mcts.search(
                    state[np.newaxis, ...], model,
                    num_simulations=mcts_simulations, add_noise=False
                )[0]
            action = sample_action(policy, temperature=0)
            state = game.next_state(state, action)

        reward = game.terminal_reward(state)
        final_player = game.current_player(state)

        if final_player == ttts_turn:
            ttts_result = reward
        else:
            ttts_result = -reward

        if ttts_result > 0:
            wins += 1
        elif ttts_result < 0:
            losses += 1
        else:
            draws += 1

    return wins, draws, losses


print("Arena function defined.")

---
## Run Ablation: Test Different backup_mode Settings

Compare:
- `visited`: Only aggregate visited children during backup (current)
- `all`: Include all children's beliefs
- `weighted`: Weight children by (visits + 1)

In [ ]:
# Load model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

game = get_game('connect4', use_rust=True)
print(f"Game backend: {game.backend}")

model_config = get_model_config(game.config, n_layer=4)
model = AlphaZeroTransformer(model_config).to(device)
load_checkpoint('checkpoints/connect4_final.pt', model)
model.eval()

# Setup PUCT baseline
puct_config = MCTSConfig()
puct_mcts = BatchedMCTS(game, puct_config)

print("Model loaded successfully.")

In [ ]:
# Ablation configuration
backup_modes = ['visited', 'all', 'weighted']
num_games = 100  # Per configuration
mcts_simulations = 100
seed = 42

print(f"\nVisited-Only Ablation")
print(f"=====================")
print(f"Games per config: {num_games}")
print(f"Simulations: {mcts_simulations}")
print(f"Config: sigma_0=1.0, obs_var=1.0")
print("=" * 50)

results = []

for mode in backup_modes:
    # Create TTTS with this backup mode
    ttts_config = BayesianMCTSConfig(
        sigma_0=1.0,
        obs_var=1.0,
    )
    ttts_mcts = BayesianMCTSVisitedOnly(game, ttts_config, backup_mode=mode)
    
    # Clear caches
    puct_mcts.clear_cache()
    ttts_mcts.clear_cache()
    
    # Set seed for reproducibility
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    # Run arena
    wins, draws, losses = run_arena(
        game, model, puct_mcts, ttts_mcts,
        num_games=num_games, mcts_simulations=mcts_simulations
    )
    
    decisive = wins + losses
    win_rate = wins / decisive if decisive > 0 else 0.5
    
    results.append({
        'mode': mode,
        'wins': wins,
        'draws': draws,
        'losses': losses,
        'win_rate': win_rate,
    })
    
    print(f"{mode:12}: {wins}W/{draws}D/{losses}L  ({win_rate:.1%} decisive win rate)")

print("\nAblation complete!")

In [ ]:
# Visualize results
import matplotlib.pyplot as plt

modes = [r['mode'] for r in results]
win_rates = [r['win_rate'] for r in results]
wins = [r['wins'] for r in results]
draws = [r['draws'] for r in results]
losses = [r['losses'] for r in results]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Win rate plot
x = np.arange(len(modes))
bars = ax1.bar(x, win_rates, color=['#2ecc71', '#3498db', '#9b59b6'], alpha=0.8)
ax1.axhline(y=0.5, color='gray', linestyle='--', label='50% baseline')
ax1.set_xticks(x)
ax1.set_xticklabels(modes)
ax1.set_xlabel('Backup Mode', fontsize=12)
ax1.set_ylabel('TTTS Win Rate vs PUCT', fontsize=12)
ax1.set_title('Win Rate by Backup Mode', fontsize=14)
ax1.set_ylim(0.3, 0.7)
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Stacked bar chart
width = 0.6

ax2.bar(x, wins, width, label='Wins', color='green', alpha=0.8)
ax2.bar(x, draws, width, bottom=wins, label='Draws', color='gray', alpha=0.8)
ax2.bar(x, losses, width, bottom=[w+d for w, d in zip(wins, draws)], label='Losses', color='red', alpha=0.8)
ax2.set_xticks(x)
ax2.set_xticklabels(modes)
ax2.set_xlabel('Backup Mode', fontsize=12)
ax2.set_ylabel('Games', fontsize=12)
ax2.set_title('Game Outcomes by Backup Mode', fontsize=14)
ax2.legend()

plt.tight_layout()
plt.savefig('visited_only_ablation.png', dpi=150)
plt.show()

# Find best mode
best_idx = np.argmax(win_rates)
best_mode = modes[best_idx]
best_win_rate = win_rates[best_idx]

print(f"\nBest backup mode: '{best_mode}' ({best_win_rate:.1%} win rate)")

---
## Extended Ablation: Varying Simulation Counts

Test if the optimal backup_mode changes with search depth.

Hypothesis:
- At low simulations (shallow search), `all` or `weighted` might help by leveraging NN priors
- At high simulations (deep search), `visited` might be better since most children are visited

In [ ]:
# Extended ablation with different sim counts
backup_modes = ['visited', 'all', 'weighted']
sim_counts = [25, 50, 100, 200]
num_games = 60  # Fewer games per cell for faster testing

print(f"\nExtended Ablation: Backup Mode x Simulation Count")
print(f"="*60)

extended_results = []

for n_sims in sim_counts:
    print(f"\n--- {n_sims} simulations ---")
    for mode in backup_modes:
        ttts_config = BayesianMCTSConfig(
            sigma_0=1.0,
            obs_var=1.0,
        )
        ttts_mcts = BayesianMCTSVisitedOnly(game, ttts_config, backup_mode=mode)
        
        puct_mcts.clear_cache()
        ttts_mcts.clear_cache()
        
        np.random.seed(seed)
        torch.manual_seed(seed)
        
        wins, draws, losses = run_arena(
            game, model, puct_mcts, ttts_mcts,
            num_games=num_games, mcts_simulations=n_sims
        )
        
        decisive = wins + losses
        win_rate = wins / decisive if decisive > 0 else 0.5
        
        extended_results.append({
            'n_sims': n_sims,
            'mode': mode,
            'wins': wins,
            'draws': draws,
            'losses': losses,
            'win_rate': win_rate,
        })
        
        print(f"  {mode:12}: {wins}W/{draws}D/{losses}L ({win_rate:.1%})")

In [ ]:
# Visualize extended results
fig, ax = plt.subplots(figsize=(10, 6))

colors = {'visited': '#2ecc71', 'all': '#3498db', 'weighted': '#9b59b6'}

for mode in backup_modes:
    mode_results = [r for r in extended_results if r['mode'] == mode]
    sims = [r['n_sims'] for r in mode_results]
    rates = [r['win_rate'] for r in mode_results]
    ax.plot(sims, rates, 'o-', linewidth=2, markersize=8, label=mode, color=colors[mode])

ax.axhline(y=0.5, color='gray', linestyle='--', label='50% baseline')
ax.set_xlabel('MCTS Simulations', fontsize=12)
ax.set_ylabel('TTTS Win Rate vs PUCT', fontsize=12)
ax.set_title('Backup Mode Effect Across Simulation Counts', fontsize=14)
ax.set_ylim(0.3, 0.7)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('visited_only_vs_sims.png', dpi=150)
plt.show()

---
## Analysis: Diagnostic Statistics

Let's examine how many children are typically unvisited at different simulation counts.

In [ ]:
# Diagnostic: How many children are unvisited?
def analyze_visit_distribution(game, model, ttts_mcts, num_games=20, n_sims=100):
    """Analyze visit distribution in TTTS search trees."""
    all_stats = []
    
    for _ in range(num_games):
        state = game.initial_state()
        ttts_mcts.clear_cache()
        
        # Just do one search
        _ = ttts_mcts.search(state[np.newaxis, ...], model, num_simulations=n_sims)
        
        # Analyze root's children
        if hasattr(ttts_mcts, '_last_roots'):  # Would need to add this
            root = ttts_mcts._last_roots[0]
        else:
            # Re-run to get root
            ttts_mcts.clear_cache()
            non_terminal_states = np.array([state])
            device = next(model.parameters()).device
            roots, _ = ttts_mcts._batch_expand_roots(non_terminal_states, model, device)
            
            # Run simulations
            for _ in range(n_sims):
                leaf_node, search_path, leaf_state, is_terminal = ttts_mcts._select_to_leaf(roots[0], state)
                if is_terminal:
                    value = game.terminal_reward(leaf_state)
                elif not leaf_node.expanded():
                    values = ttts_mcts._batch_expand_leaves([leaf_node], [leaf_state], model, device)
                    value = values[0]
                else:
                    value = leaf_node.mu
                ttts_mcts._backup(search_path, value)
            
            root = roots[0]
        
        if root.expanded():
            children = list(root.children.values())
            visits = [c.visits for c in children]
            n_children = len(children)
            n_visited = sum(1 for v in visits if v > 0)
            n_unvisited = n_children - n_visited
            
            all_stats.append({
                'n_children': n_children,
                'n_visited': n_visited,
                'n_unvisited': n_unvisited,
                'pct_unvisited': n_unvisited / n_children if n_children > 0 else 0,
                'max_visits': max(visits) if visits else 0,
                'min_visits': min(visits) if visits else 0,
            })
    
    return all_stats


# Run diagnostics at different simulation counts
print("Visit Distribution Analysis")
print("=" * 60)

for n_sims in [25, 50, 100, 200]:
    ttts_config = BayesianMCTSConfig(sigma_0=1.0, obs_var=1.0)
    ttts_mcts = BayesianMCTSVisitedOnly(game, ttts_config, backup_mode='visited')
    
    stats = analyze_visit_distribution(game, model, ttts_mcts, num_games=20, n_sims=n_sims)
    
    avg_children = np.mean([s['n_children'] for s in stats])
    avg_unvisited = np.mean([s['n_unvisited'] for s in stats])
    avg_pct_unvisited = np.mean([s['pct_unvisited'] for s in stats])
    
    print(f"n_sims={n_sims:3d}: avg {avg_children:.1f} children, "
          f"{avg_unvisited:.1f} unvisited ({avg_pct_unvisited:.1%})")

---
## Summary

### Backup Mode Options

| Mode | Description | When it might help |
|------|-------------|--------------------|
| `visited` | Only aggregate visited children | Default; avoids noise from unvisited priors |
| `all` | Include all children's beliefs | Shallow search with informative NN priors |
| `weighted` | Weight by (visits + 1) | Compromise; trusted moves weighted more |

### Interpretation

1. **If `visited` is best**: Unvisited children's priors add noise; actual search results are more reliable
2. **If `all` is best**: NN priors are informative enough that including them helps
3. **If `weighted` is best**: Priors help but should be down-weighted vs visited children
4. **If optimal varies with n_sims**: Consider adaptive strategy based on search depth

### Next Steps Based on Results

If a non-default mode performs better:
1. Add `backup_mode` parameter to `BayesianMCTSConfig`
2. Update `_backup()` method to use the configured mode
3. Consider making it adaptive based on simulation count or tree depth